In [1]:
import os

os.environ['NVIDIA_API_KEY'] = "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"
from omegaconf import OmegaConf
import sys
sys.path.append("../")
sys.path.append("../../../")
from retriever_evalset_generator import RetrieverEvalSetGenerator
from filters import EasinessFilter, AnswerabilityFilter
from nemo_curator.modules.filter import ScoreFilter, Score
from nemo_curator.datasets import DocumentDataset
from nemo_curator.modules.config import RetrieverEvalSDGConfig
import warnings
warnings.filterwarnings('ignore')

import dask.dataframe as dd
from distributed import Client
from distributed.diagnostics.progressbar import progress
from tqdm import tqdm


/home/vinay/.local/lib/python3.10/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
import pandas as pd
df = pd.read_json("../data/nq_test.jsonl", lines=True)
df = df[:5]

In [3]:
cfg = RetrieverEvalSDGConfig.from_yaml("../config/retriever_evals_sdg.yaml")
retrieval_evalset_generator = RetrieverEvalSetGenerator(cfg)

In [4]:

dataset = DocumentDataset.from_pandas(df)
generated_dataset = retrieval_evalset_generator(dataset)


In [5]:
generated_dataset.df.head(n=5, npartitions=-1)


,_id,text,title,question-id,question,answer,score
0,doc6,"The fourth season of Chicago Fire, an American...",Chicago Fire (season 4),fd8eac5f9bc486aed28011766672618b5c08cb2328aac7...,Who are the executive producer and producers o...,The executive producer of the fourth season of...,1
1,doc6,"The fourth season of Chicago Fire, an American...",Chicago Fire (season 4),72c257e6653078e9260865913c6b2c35fbcd9bcd182578...,When did the fourth season of Chicago Fire pre...,The fourth season of Chicago Fire premiered on...,1
2,doc6,"The fourth season of Chicago Fire, an American...",Chicago Fire (season 4),d03950f89e73227a6ae816a9de1e7d8a0cbf174f09607c...,How many episodes were there in the fourth sea...,There were 23 episodes in the fourth season of...,1
3,doc10,"""Love Will Keep Us Alive"" is a song written by...",Love Will Keep Us Alive,e505c0858cbe44d6eff2be3a66f1ce4f8c031504f6605b...,"Who are the songwriters of ""Love Will Keep Us ...","""Love Will Keep Us Alive"" was written by Jim C...",1
4,doc10,"""Love Will Keep Us Alive"" is a song written by...",Love Will Keep Us Alive,b4b6b2169318f42db87742a58cc034617b9f0e733a885e...,"When was ""Love Will Keep Us Alive"" first perfo...","""Love Will Keep Us Alive"" was first performed ...",1


In [6]:
ef = EasinessFilter(cfg)
easiness_filter = ScoreFilter(ef,
                              text_field = ["text", "question"],
                              score_field = "easiness_scores")
af = AnswerabilityFilter(cfg)
answerability_filter = ScoreFilter(af,
                              text_field = ["text", "question"],
                              score_field = "answerability_scores")

In [7]:
filtered_dataset = easiness_filter(generated_dataset)
filtered_dataset.df.head(n=5, npartitions=-1)

,_id,text,title,question-id,question,answer,score,easiness_scores
0,doc6,"The fourth season of Chicago Fire, an American...",Chicago Fire (season 4),f794dcfdb910a120ab6105af27bccc2d4dcfc3dfb972b0...,Who are the executive producer and producers o...,The executive producer of the fourth season of...,1,0.621422
3,doc10,"""Love Will Keep Us Alive"" is a song written by...",Love Will Keep Us Alive,09414ba224e94622cf3cf989d266a2b52ac1677c9ee3e3...,"Who are the songwriters of ""Love Will Keep Us ...","""Love Will Keep Us Alive"" was written by Jim C...",1,0.583219
4,doc10,"""Love Will Keep Us Alive"" is a song written by...",Love Will Keep Us Alive,4474e52d80e31df757e4f9ff78eaba2c7394e66533a920...,"When was ""Love Will Keep Us Alive"" first perfo...","""Love Will Keep Us Alive"" was first performed ...",1,0.632935
6,doc42,"""Fishin' in the Dark"" is a song written by Wen...",Fishin' in the Dark,e0d200329a8a3e70a61a7f377e6c87b4167da2c4647565...,"Who wrote the song ""Fishin' in the Dark""?","""Fishin' in the Dark"" was written by Wendy Wal...",1,0.645659
7,doc42,"""Fishin' in the Dark"" is a song written by Wen...",Fishin' in the Dark,df3bd5e7bf6d18053d9e61d22d321d85db0847423cf944...,"Which American country music group recorded ""F...","""Fishin' in the Dark"" was recorded by American...",1,0.660160


In [8]:
filtered_dataset_2 = answerability_filter(filtered_dataset)
filtered_dataset_2.df.head(n=5, npartitions=-1)

,_id,text,title,question-id,question,answer,score,easiness_scores,answerability_scores
0,doc6,"The fourth season of Chicago Fire, an American...",Chicago Fire (season 4),56df30ad2e334441f829611468a447cee83813cffc67e7...,Who are the executive producer and producers o...,The executive producer of the fourth season of...,1,0.621422,"{\n""criterion_1_explanation"": ""The question is..."
3,doc10,"""Love Will Keep Us Alive"" is a song written by...",Love Will Keep Us Alive,09e45864be86a3a506d3a7749b013529dbdb1c26377991...,"Who are the songwriters of ""Love Will Keep Us ...","""Love Will Keep Us Alive"" was written by Jim C...",1,0.583219,"{\n""criterion_1_explanation"": ""The question is..."
4,doc10,"""Love Will Keep Us Alive"" is a song written by...",Love Will Keep Us Alive,e9a71e6ea11a84f8728e8b30420f510598f411891e48dc...,"When was ""Love Will Keep Us Alive"" first perfo...","""Love Will Keep Us Alive"" was first performed ...",1,0.632935,"{\n""criterion_1_explanation"": ""The question is..."
6,doc42,"""Fishin' in the Dark"" is a song written by Wen...",Fishin' in the Dark,c5818e95a1e9fc17759854dd114194513195597615d2fa...,"Who wrote the song ""Fishin' in the Dark""?","""Fishin' in the Dark"" was written by Wendy Wal...",1,0.645659,"{\n""criterion_1_explanation"": ""The question is..."
7,doc42,"""Fishin' in the Dark"" is a song written by Wen...",Fishin' in the Dark,8a9abc14b96ad1b74b809df46806c131165fa4027451c3...,"When was ""Fishin' in the Dark"" released as a s...","""Fishin' in the Dark"" was released as a single...",1,0.636784,"{\n""criterion_1_explanation"": ""The question is..."
